#### Code inspired by https://github.com/dibondar/QControlPolyOpt/tree/master

In [1]:
using TSSOS
using LinearAlgebra
using QuantumOptics
using DynamicPolynomials
using QuadGK
using JuMP
using Random
using NLopt

## Define the quantum system

In [2]:
using LinearAlgebra

I2 = Matrix{Float64}(I, 2, 2)
X = [0.0 1.0; 1.0 0.0]
Z = [1.0 0.0; 0.0 -1.0]

function kron_n_single(n::Int, op::Matrix{Float64}, pos::Int)
    ops = [I2 for _ in 1:n]
    ops[pos] = op
    return foldl(kron, ops)
end

function kron_n_double(n::Int, op1, pos1, op2, pos2)
    ops = [I2 for _ in 1:n]
    ops[pos1] = op1
    ops[pos2] = op2
    return foldl(kron, ops)
end


function H0_Ising(n::Int)
    dim = 2^n
    H0 = zeros(Float64, dim, dim)
    for i in 1:n-1
        H0 .+= kron_n_double(n, Z, i, Z, i+1)
    end
    return H0
end

function Hc_Ising(n::Int)
    dim = 2^n
    Hc = zeros(Float64, dim, dim)
    for i in 1:n
        Hc .+= kron_n_single(n, X, i)
    end
    return Hc
end


Hc_Ising (generic function with 1 method)

## Useful functions with polynomials

In [4]:
function ∫(p::AbstractPolynomial, x::PolyVar, x_lower, x_upper)
    
    # get the index of the variable of integration
    ind_x = indexin([x], variables(p))[1]
        
    if isnothing(ind_x)
        # integration valuable is not found among vars
        return p * (x_upper - x_lower)
    end
    
    # get the indefinite integral
    int_p = sum(
        term * x * 1 // (exponents(term)[ind_x] + 1) for term in terms(p)
        init = 0 * x
    )
            
    # get the definite integral
    subs(int_p, x=>x_upper) - subs(int_p, x=>x_lower)
end

function ∫(M::AbstractMatrix, x::PolyVar, x_lower, x_upper)
   map(z -> ∫(z, x, x_lower, x_upper), M) 
end

function real_poly(p::Polynomial)
    #=
    Real part of the polynomial
    =#
    sum(
        real(c) * m for (c, m) in zip(coefficients(p), monomials(p))# if ~isapproxzero(abs(c))
    )
end

function square_frobenius_norm(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    real_poly(sum(z' * z for z in M))
end

function square_frobenius_norm2(M::AbstractArray)
    #=
    Square of the Frobenius norm of a matrix
    =#
    sum(z' * z for z in M)
end

square_frobenius_norm2 (generic function with 1 method)

## Magnus expansion

In [5]:
@polyvar x[1:3]
@polyvar t[1:3]

# final time
const T = 0.5

function u(t, x)
    # the polynomial shape for control
    sum(x[n] * t^(n - 1) for n = 1:length(x))
end

function Magnus(n)
    function A(t, x, n)
        #=
        The generator of motion entering the Magnus expansion
        =#
        H0 = H0_Ising(n)
        Hc = Hc_Ising(n)
        (H0 + Hc * u(t, x)) / im
    end
    
    function commutator(a, b)
        a * b - b * a
    end 
    
    # get the partial sum of the Magnus expansion
    A₁ = A(t[1], x, n)
    A₂ = A(t[2], x, n)
    
    Ω = ∫(A₁, t[1], 0, T);
    
    # 2nd term in the Magnus expansion
    Ω .+= 1//2 * ∫(∫(
        commutator(A₁, A₂), 
        t[2], 0, t[1]), 
        t[1], 0, T
    );
    
    # 3nd term in the Magnus expansion
    
    A₃ = A(t[3], x, n)
    
    Ω .+= 1//6 * ∫(∫(∫(
        commutator(A₁, commutator(A₂, A₃)) + commutator(commutator(A₁, A₂), A₃),
        t[3], 0, t[2]),
        t[2], 0, t[1]),
        t[1], 0, T
    );

    return Ω
end


Magnus (generic function with 1 method)

## Get random unitary target

In [6]:
Random.seed!(65)
exact_x = -1 .+ 2 * rand(3)

function get_unitary(x::AbstractArray, n)
    #=
    Get the unitary given the coefficients for the polynomial control
    =#
    H0 = H0_Ising(n)
    Hc = Hc_Ising(n)
    
    basis = NLevelBasis(size(H0)[1])

    𝓗₀ = DenseOperator(basis, basis, H0)
    𝓥 = DenseOperator(basis, basis, Hc)

    H = LazySum([1., u(0, x)], [𝓗₀, 𝓥])
        
    function 𝓗(t, psi)
        H.factors[2] = u(t, x)
        return H
    end

    _, 𝓤 = timeevolution.schroedinger_dynamic([0, T], identityoperator(basis, basis), 𝓗)
    
    return Matrix(𝓤[2].data)
end

get_unitary (generic function with 1 method)

In [7]:
exact_x

3-element Vector{Float64}:
 -0.1874836569137377
 -0.8380629706479825
 -0.3946330275188754

## Compare for multiple qubits

In [8]:
Random.seed!(2801)

for j in [2,3,4,5]

    Ωj = Magnus(j)
    n_samples = 10
    
    
    # randomly generate the coefficients for the polynomial control 
    exact_x = -1 .+ 2 * rand(length(x) * n_samples)
    exact_x = reshape(exact_x, (length(x), n_samples))
        
    # The array of target unitaries synthesized by the control x
    U_targets = zeros(ComplexF64, n_samples, size(H0_Ising(j))...)
    
    # The normalised overlap of the evolution and the target 
    inf = zeros(n_samples) 
    
    for i=1:n_samples
        
        # target unitray
        U_targets[i, :, :] = U_target = get_unitary(exact_x[:, i], j)
            
        # get the polynomial objective function
        obj = square_frobenius_norm(
            Ωj - log(U_targets[i,:,:])
        )
      
        
        # Get the global minimum via TSSOS library
        opt,sol,data = tssos_first(obj, variables(obj); QUIET = true, solution = true)
        
        previous_sol = sol
        previous_opt = opt
        
        while ~isnothing(sol)
            previous_sol = sol
            previous_opt = opt
                
            opt,sol,data = tssos_higher!(data; QUIET = true, solution = true)
        end
    
        # get the Frobenius norm difference between target and obtained unitaries
        U_star = get_unitary(previous_sol, j)
    
        # The normalised overlap of the evolution and the target 
        inf[i] = abs(tr(U_target' * U_star)) / size(U_star)[1]
        
    
    end

    using HDF5
    
    
    h5open("results_$(j)_10.hdf5", "w") do fid
        fid["U_targets"] = U_targets
        fid["exact_x"] = exact_x
        fid["Infidelity"] = inf
    end;
    
end

*********************************** TSSOS ***********************************
TSSOS is launching...
optimum = 1.3731589723208696e-8
Global optimality certified with relative optimality gap 0.000001%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
TSSOS is launching...
optimum = 1.1360691305887292e-8
Global optimality certified with relative optimality gap 0.000001%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
TSSOS is launching...
optimum = 1.1766373124494511e-8
Global optimality certified with relative optimality gap 0.000001%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
TSSOS is launching...
optimum = 1.7449110533846957e-8
Global optimality certified with relative optimality gap 0.000002%!
No higher TS step of the TSSOS hierarchy!
********************************